In [ ]:
import pandas as pd
file = pd.read_csv('events_up_to_01062018.csv')

In [ ]:
file['timestamp'] = pd.to_datetime(file['timestamp'])

In [29]:
def cantidades (x):
    compras = (x['event']=='conversion').sum()
    visitas = (x['event']=='visited site').sum()
    cant_prod_vistos = (x['event']=='viewed product').sum()
    return pd.Series([compras,visitas,cant_prod_vistos])
cantidades = file.groupby('person').apply(cantidades)
cantidades.columns=['compras','visitas','cant_prod_vistos']

In [30]:
def dispos (x):
    dispos = x['device_type'].value_counts().index
    if dispos.size>0:
        dispositivo = dispos[0]
    else:
        dispositivo = 'Ninguno'
    return pd.Series([dispositivo])
dispos = file.groupby('person').apply(dispos)
dispos.columns=['dispos']

In [31]:
def prodvisto (x):
    prods = x.loc[x.event=='viewed product','model'].value_counts().index
    if prods.size>0:
        prod_visto = prods[0]
    else:
        prod_visto = 'Ninguno'
    return pd.Series([prod_visto])
prodvisto = file.groupby('person').apply(prodvisto)
prodvisto.columns = ['prod_visto']

In [32]:
def mayorevento (x):
    mayor_evento = x['event'].value_counts().index[0]
    return pd.Series([mayor_evento])
mayorevento = file.groupby('person').apply(mayorevento)
mayorevento.columns = ['mayor_evento']

In [33]:
features = mayorevento.join(prodvisto).join(dispos).join(cantidades)

In [34]:
def temporal (x):
    s = x['timestamp'].size
    primer_visita = x['timestamp'].min()
    ultima_visita = x['timestamp'].max()
    cant_eventos = s
    return pd.Series([primer_visita,ultima_visita,cant_eventos])
temporal = file.groupby('person').apply(temporal)

In [35]:
def channel (x):
    channels = x['channel'].value_counts()
    if channels.size > 0:
        return channels.index[0]
    else:
        return 'Ninguno'
channel = file.groupby('person').apply(channel)
channel.columns = ['channel']

In [36]:
temporal.columns = ['primer_visita','ultima_visita','cant_eventos']
processed = features.join(temporal)

In [37]:
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos
person,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17


In [38]:
processed['permanencia'] = (processed['ultima_visita']-processed['primer_visita']).dt.days + 1

In [39]:
processed['hace_cuanto_visito'] = (file['timestamp'].max()-processed['ultima_visita']).dt.days

In [40]:
processed['eventos_por_dia'] = processed['cant_eventos']/processed['permanencia']

In [41]:
processed['channel'] = channel

In [42]:
processed['visitas_por_dia'] = processed['visitas']/processed['permanencia']

In [ ]:
processed.to_csv('processed.csv')